# K-Means Clustering — Warmup Activity

This warm-up activity demonstrates how a the k-means clustering algorithm automatically finds the centers of clusters in the data and assigns points to groups.

We begin, as usual, by importing the python libraries we need and setting a few matplotlib plotting options.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from datascience import Table

# Plot style
plt.rcParams['figure.figsize'] = (5, 5)
plt.rcParams['axes.grid'] = True

%matplotlib inline

# Reproducibility
# rng stands for "random number generator"
rng = np.random.default_rng(42)

# Distinct color palette (matplotlib 'tab10')
TAB10 = np.array([
    '#1f77b4','#ff7f0e','#2ca02c','#d62728','#9467bd',
    '#8c564b','#e377c2','#7f7f7f','#bcbd22','#17becf'
])

## Create a small data set with three clusters of points. 
Each cluster is comprised of points assigned randomly around different center points.

In [ ]:
# Make a small 2-D dataset (3 "blobs")
n_per = 30
blob1 = rng.normal([0.0, 0.0], 0.40, (n_per, 2))
blob2 = rng.normal([3.0, 0.5], 0.50, (n_per, 2))
blob3 = rng.normal([0.5, 3.0], 0.45, (n_per, 2))
X = np.vstack([blob1, blob2, blob3])
tbl = Table().with_columns("x", X[:,0], "y", X[:,1])
tbl

In [ ]:
# Quick look at the data
tbl.scatter("x", "y", s=20)
plt.title(f"Toy dataset (n={tbl.num_rows})")
plt.show()

## Student Challenge 1:
Looking at the code, what are the true x, y coordinates of the centers of the three clusters?

In [ ]:
def distance(p, c):
    """Euclidean distance between two 2-D points p and c."""
    dx = p[0] - c[0]
    dy = p[1] - c[1]
    return (dx*dx + dy*dy)**0.5

## Student Challenge 2:
Explain what the `distance` function does.

In [ ]:
def assign_labels_loop(points, centers):
    """For each point, find the index of its nearest center (plain loops)."""
    labels = []
    for p in points:
        dists = [distance(p, c) for c in centers]
        labels.append(int(np.argmin(dists)))
    return np.array(labels)

In [ ]:
labels = assign_labels_loop(X, centers)
labels

## Student Challenge 3: 
Explain how the `assign_labels_loop` function works.

In [ ]:
def update_centers_loop(points, labels, k):
    """Recompute each center as the mean of its assigned points."""
    new_centers = []
    for j in range(k):
        members = points[labels == j]
        if len(members) > 0:
            new_centers.append(members.mean(axis=0))
        else:
            # If a cluster is empty, keep the old center or reseed;
            # here we keep NaN to make it obvious in class if it happens.
            new_centers.append(np.array([np.nan, np.nan]))
    return np.array(new_centers)

### Take some time to understand how this last function works
The `update_centers_loop` function is the heart of the k-means algorithm because it’s where the algorithm moves the centers toward the “middle” of their assigned points. 

`new_center = []` \
We’ll fill this list with the updated coordinates of each cluster center.

`for j in range(k):`\
The variable k is the number of clusters (e.g., 3 if you’re doing 3-means).\
So the loop runs once for each cluster.

`members = points[labels == j]`\
	•	points is a 2-D NumPy array with shape (n_points, 2) (x, y coordinates).\
	•	labels is a 1-D array of integers where labels[i] is the cluster index for point i.

`points[labels == j]`\
selects only the rows of points whose label equals j.
→ members is an array of all points currently assigned to cluster j.

`if len(members) > 0:`\
If the cluster has at least one member
 
`new_centers.append(members.mean(axis=0))`\
 take the mean of all its x and y coordinates. This moves the center to the geometric middle of its points.

 `else:`
  What about empty clusters?
  
`new_centers.append(np.array([np.nan, np.nan]))`
Sometimes a cluster ends up with no points (for example, if all points were closer to some other center).
Here, we add a placeholder [NaN, NaN] so we keep the same number of centers. (NaN is short for 'not a number')

`return np.array(new_centers)`
Return all new centers

#### Summary
For each cluster:

1.	Collect all its points.
2.	Compute the average position (the centroid).
3.	Replace the old center with this new average.

That’s the “update” step in the “assign–update” loop that defines k-means.

In [ ]:
def sse(points, centers, labels):
    """Sum of squared errors to assigned centers (lower is better)."""
    total = 0.0
    for p, lab in zip(points, labels):
        c = centers[lab]
        dx = p[0] - c[0]
        dy = p[1] - c[1]
        total += dx*dx + dy*dy
    return total

## Student Challenge #4:
Explain what the `sse` function does.

In [ ]:
def update_plot(tbl, centers, labels, point_size=36, alpha=0.85, show_legend=True):
    """
    Draw the clusters with distinct colors and the centers as black X's.
    tbl: datascience.Table with 'x1','x2'
    centers: (k,2) array
    labels:  length-n array of cluster indices per point
    """
    X = np.column_stack([tbl.column("x"), tbl.column("y")])
    k = len(centers)

    plt.figure(figsize=(5,5))
    for j in range(k):
        pts = X[labels == j]
        if len(pts) == 0:
            continue
        plt.scatter(pts[:,0], pts[:,1],
                    c=TAB10[j % len(TAB10)],
                    s=point_size, alpha=alpha, label=f"Cluster {j}")

    plt.scatter(centers[:,0], centers[:,1],
                c='black', marker='X', s=180,
                edgecolor='white', linewidth=1.2,
                label='Centers')

    if show_legend:
        plt.legend(frameon=True)
    plt.xlabel("x"); plt.ylabel("y")
    plt.title("K-means (assignment + current centers)")
    plt.grid(True, alpha=0.25)
    plt.show()

## Initialize with random cluster centers
At the start of the algorithm we just randomly guess the centers of the clusters and we tell the algorithm how many cluster to look for.  Changing k changes the number of clusters.

All of the points are then labeled with the number of the cluster center they are closest to and the labels are used to chose the colors of the points in the scatter plot.

In [ ]:
k = 3  # try 2, 3, 4
init_idx = rng.choice(len(X), size=k, replace=False)
centers = X[init_idx].copy()

print("Initial centers:\n", centers)

# First assignment + plot
labels = assign_labels_loop(X, centers)
update_plot(tbl, centers, labels)
print(f"SSE: {sse(X, centers, labels):.2f}")

## Not particularly good centers
The initial guesses don't find the best clusters. This is were the magic starts. We use the 'update_centers_loop' function to keep moving the centers, each time reducing sum of squared errors, looking for the centers around which the points in each group cluster most tightly.

**The cell below runs three updates. Look at how the centers move from plot to plot.**

In [ ]:
# Run a few manual iterations so students can see movement
num_steps = 3   # try 1, 2, 3

for step in range(1, num_steps+1):
    labels = assign_labels_loop(X, centers)
    centers = update_centers_loop(X, labels, k)
    print(f"After step {step}: SSE = {sse(X, centers, labels):.2f}")
    update_plot(tbl, centers, labels)

## Student Challenge #5:
At the start, we told the algorithm to look for 3 clusters. Try changing k to 5.
What happens in this case.

## Student Challenge #6:
Describe a possible real-world application for an algorithm that finds clusters. Don't Google, think about it!